In [3]:
from tpot import TPOTClassifier

In [7]:
!pip install wwo-hist

  Using cached wwo_hist-0.0.7-py3-none-any.whl (4.6 kB)


In [14]:
import json
from wwo_hist import retrieve_hist_data
import pandas as pd
import pickle
import sklearn

## 1. Location Name - density, latitude, longitude

In [15]:
def get_data(location_name):
    json_data = json.load(open('update_data.json','r'))
    for data in json_data:
        if data["location"] in location_name:
            lat = data["latitude"]
            log = data["longitude"]
            den = data["density_per_km"]
            
    return den, lat, log

In [16]:
latlog_data = get_data("Argentina-Buenos_Aires")
latlog_data

(12625, -34.6036844, -58.3815591)

## 2. weather data

In [17]:
def get_wheather_data(location_name,start_date,end_date):
    
    start_date = pd.to_datetime(start_date, format="%Y-%m-%d")
    start_date = start_date - pd.to_timedelta(7, unit='d')
    start_date = start_date.strftime('%Y-%m-%d')
    
    end_date = pd.to_datetime(end_date, format="%Y-%m-%d")
    end_date = end_date - pd.to_timedelta(7, unit='d')
    end_date = end_date.strftime('%Y-%m-%d')
    
    frequency = 24
    api_key = '092a2ffeb04645c5ba855736210909' 

    location_list = [location_name]
    hist_weather_data = retrieve_hist_data(api_key,
                                    location_list,
                                    start_date,
                                    end_date,
                                    frequency,
                                    location_label = False,
                                    export_csv=False,
                                    store_df = True)
    data = hist_weather_data[0]
    
    data[['maxtempC','mintempC', 'totalSnow_cm', 'sunHour', 'moon_illumination', 'DewPointC',
       'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'visibility', 'winddirDegree']] = data[['maxtempC',
       'mintempC', 'totalSnow_cm', 'sunHour', 'moon_illumination', 'DewPointC',
       'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'visibility', 'winddirDegree']].apply(pd.to_numeric)
        
    return data

In [18]:
weather_data = get_wheather_data('Argentina-Buenos_Aires','2016-02-04','2016-02-11')
weather_data



Retrieving weather data for Argentina-Buenos_Aires


Currently retrieving data for Argentina-Buenos_Aires: from 2016-01-28 to 2016-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.266377
Currently retrieving data for Argentina-Buenos_Aires: from 2016-02-01 to 2016-02-04
Time elapsed (hh:mm:ss.ms) 0:00:02.707511


,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2016-01-28,28,15,0.0,11.6,6,69,10:44 PM,10:22 AM,06:06 AM,...,25,28,49,0.2,1014,28,10,177,17,Argentina-Buenos_Aires
0,2016-01-29,32,15,0.0,10.5,6,61,11:15 PM,11:17 AM,06:07 AM,...,15,20,52,1.0,1013,32,10,176,9,Argentina-Buenos_Aires
0,2016-01-30,30,17,0.0,11.6,6,54,11:47 PM,12:11 PM,06:09 AM,...,24,19,65,2.5,1015,30,10,160,17,Argentina-Buenos_Aires
0,2016-01-31,25,10,0.0,11.6,4,50,No moonrise,01:06 PM,06:10 AM,...,29,14,63,0.0,1020,25,9,167,20,Argentina-Buenos_Aires
0,2016-02-01,28,11,0.0,11.6,5,34,12:20 AM,02:00 PM,06:11 AM,...,21,20,65,0.0,1021,28,8,130,13,Argentina-Buenos_Aires
0,2016-02-02,30,13,0.0,11.6,5,26,12:56 AM,02:56 PM,06:12 AM,...,18,10,59,0.0,1018,30,8,42,11,Argentina-Buenos_Aires
0,2016-02-03,33,15,0.0,11.6,6,18,01:37 AM,03:51 PM,06:13 AM,...,26,29,55,0.0,1012,33,9,70,16,Argentina-Buenos_Aires
0,2016-02-04,33,17,0.0,10.4,6,11,02:21 AM,04:45 PM,06:14 AM,...,22,25,50,0.3,1010,33,10,132,13,Argentina-Buenos_Aires


In [19]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 0
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date_time          8 non-null      datetime64[ns]
 1   maxtempC           8 non-null      int64         
 2   mintempC           8 non-null      int64         
 3   totalSnow_cm       8 non-null      float64       
 4   sunHour            8 non-null      float64       
 5   uvIndex            8 non-null      object        
 6   moon_illumination  8 non-null      int64         
 7   moonrise           8 non-null      object        
 8   moonset            8 non-null      object        
 9   sunrise            8 non-null      object        
 10  sunset             8 non-null      object        
 11  DewPointC          8 non-null      int64         
 12  FeelsLikeC         8 non-null      object        
 13  HeatIndexC         8 non-null      object        
 14  WindChillC    

# 1 Classification 

In [20]:
def data_for_clf(weather_data,latlog_data):

    # create new year and month column from datetime
    weather_data['year'] = pd.DatetimeIndex(weather_data['date_time']).year
    weather_data['month'] = pd.DatetimeIndex(weather_data['date_time']).month
    weather_data = weather_data.drop(['uvIndex','location','moonrise', 
                                      'moonset', 'sunrise','sunset','FeelsLikeC', 
                                      'HeatIndexC', 'WindChillC','tempC','windspeedKmph'], axis = 1)
    
    weather_data['density_per_km'] = latlog_data[0]
    weather_data['latitude'] = latlog_data[1]
    weather_data['longitude'] = latlog_data[2]
    weather_data = weather_data[['date_time', 'density_per_km', 'latitude', 'longitude', 'maxtempC', 
                                 'mintempC', 'totalSnow_cm', 'sunHour','moon_illumination', 'DewPointC',
                                 'WindGustKmph', 'cloudcover','humidity', 'precipMM', 'pressure', 
                                 'visibility', 'winddirDegree','year', 'month']]
    return weather_data

In [21]:
clf_data = data_for_clf(weather_data,latlog_data)
clf_data

,date_time,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month
0,2016-01-28,12625,-34.603684,-58.381559,28,15,0.0,11.6,69,9,25,28,49,0.2,1014,10,177,2016,1
0,2016-01-29,12625,-34.603684,-58.381559,32,15,0.0,10.5,61,11,15,20,52,1.0,1013,10,176,2016,1
0,2016-01-30,12625,-34.603684,-58.381559,30,17,0.0,11.6,54,14,24,19,65,2.5,1015,10,160,2016,1
0,2016-01-31,12625,-34.603684,-58.381559,25,10,0.0,11.6,50,9,29,14,63,0.0,1020,9,167,2016,1
0,2016-02-01,12625,-34.603684,-58.381559,28,11,0.0,11.6,34,10,21,20,65,0.0,1021,8,130,2016,2
0,2016-02-02,12625,-34.603684,-58.381559,30,13,0.0,11.6,26,11,18,10,59,0.0,1018,8,42,2016,2
0,2016-02-03,12625,-34.603684,-58.381559,33,15,0.0,11.6,18,12,26,29,55,0.0,1012,9,70,2016,2
0,2016-02-04,12625,-34.603684,-58.381559,33,17,0.0,10.4,11,12,22,25,50,0.3,1010,10,132,2016,2


In [22]:
clf_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 0
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date_time          8 non-null      datetime64[ns]
 1   density_per_km     8 non-null      int64         
 2   latitude           8 non-null      float64       
 3   longitude          8 non-null      float64       
 4   maxtempC           8 non-null      int64         
 5   mintempC           8 non-null      int64         
 6   totalSnow_cm       8 non-null      float64       
 7   sunHour            8 non-null      float64       
 8   moon_illumination  8 non-null      int64         
 9   DewPointC          8 non-null      int64         
 10  WindGustKmph       8 non-null      int64         
 11  cloudcover         8 non-null      int64         
 12  humidity           8 non-null      int64         
 13  precipMM           8 non-null      float64       
 14  pressure      

# 1.1 Model predict

In [23]:
def classifier_model(data):
    classifier_model = pickle.load(open("classifier_model.pkl","rb"))
    predict_data = data.drop(['date_time'], axis = 1)
    result = classifier_model.predict_proba(predict_data)
    nozika = [x[0] for x in result]
    zika = [x[1] for x in result]
    data['Zika'] = zika
    data['No Zika'] = nozika

    return data

In [26]:
# 0 Nozika and 1 zika
clf_result = classifier_model(clf_data)
clf_result

,date_time,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,...,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month,Zika,No Zika
0,2016-01-28,12625,-34.603684,-58.381559,28,15,0.0,11.6,69,9,...,28,49,0.2,1014,10,177,2016,1,0.287580,0.712420
0,2016-01-29,12625,-34.603684,-58.381559,32,15,0.0,10.5,61,11,...,20,52,1.0,1013,10,176,2016,1,0.289510,0.710490
0,2016-01-30,12625,-34.603684,-58.381559,30,17,0.0,11.6,54,14,...,19,65,2.5,1015,10,160,2016,1,0.318075,0.681925
0,2016-01-31,12625,-34.603684,-58.381559,25,10,0.0,11.6,50,9,...,14,63,0.0,1020,9,167,2016,1,0.198488,0.801512
0,2016-02-01,12625,-34.603684,-58.381559,28,11,0.0,11.6,34,10,...,20,65,0.0,1021,8,130,2016,2,0.251371,0.748629
0,2016-02-02,12625,-34.603684,-58.381559,30,13,0.0,11.6,26,11,...,10,59,0.0,1018,8,42,2016,2,0.234247,0.765753
0,2016-02-03,12625,-34.603684,-58.381559,33,15,0.0,11.6,18,12,...,29,55,0.0,1012,9,70,2016,2,0.297792,0.702208
0,2016-02-04,12625,-34.603684,-58.381559,33,17,0.0,10.4,11,12,...,25,50,0.3,1010,10,132,2016,2,0.299516,0.700484


# 2. Regression

In [27]:
def data_for_rg(weather_data,latlog_data):

    # create new year and month column from datetime
    weather_data['year'] = pd.DatetimeIndex(weather_data['date_time']).year
    weather_data['month'] = pd.DatetimeIndex(weather_data['date_time']).month
    
    weather_data = weather_data.drop(['totalSnow_cm',
       'uvIndex', 'moon_illumination', 'moonrise', 'moonset', 'sunrise',
       'sunset', 'HeatIndexC', 'WindChillC',
       'WindGustKmph', 'pressure','tempC', 'visibility', 'location'], axis = 1)
    
    weather_data['density_per_km'] = latlog_data[0]
    weather_data['latitude'] = latlog_data[1]
    weather_data['longitude'] = latlog_data[2]
    weather_data = weather_data[['date_time','maxtempC', 'mintempC', 'windspeedKmph', 'year', 'month', 'density_per_km', 'precipMM', 'cloudcover',
                                   'humidity','DewPointC', 'latitude', 'longitude','FeelsLikeC', 'winddirDegree', 'sunHour']]
    return weather_data

In [28]:
reg_data = data_for_rg(weather_data,latlog_data)
reg_data

,date_time,maxtempC,mintempC,windspeedKmph,year,month,density_per_km,precipMM,cloudcover,humidity,DewPointC,latitude,longitude,FeelsLikeC,winddirDegree,sunHour
0,2016-01-28,28,15,17,2016,1,12625,0.2,28,49,9,-34.603684,-58.381559,21,177,11.6
0,2016-01-29,32,15,9,2016,1,12625,1.0,20,52,11,-34.603684,-58.381559,22,176,10.5
0,2016-01-30,30,17,17,2016,1,12625,2.5,19,65,14,-34.603684,-58.381559,23,160,11.6
0,2016-01-31,25,10,20,2016,1,12625,0.0,14,63,9,-34.603684,-58.381559,17,167,11.6
0,2016-02-01,28,11,13,2016,2,12625,0.0,20,65,10,-34.603684,-58.381559,18,130,11.6
0,2016-02-02,30,13,11,2016,2,12625,0.0,10,59,11,-34.603684,-58.381559,21,42,11.6
0,2016-02-03,33,15,16,2016,2,12625,0.0,29,55,12,-34.603684,-58.381559,23,70,11.6
0,2016-02-04,33,17,13,2016,2,12625,0.3,25,50,12,-34.603684,-58.381559,25,132,10.4


# 2.2 Model Prediction

In [29]:
def regressor_model(data):
    regressor_model = pickle.load(open("regression_model.pkl","rb"))
    predict_data = data.drop(['date_time'], axis = 1)
    result = regressor_model.predict(predict_data)
    return result

In [30]:
reg_result = regressor_model(reg_data)
reg_result

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:    1.2s finished


array([131.16897993,  96.43905099, 124.9360141 ,  87.51267286,
        65.60643922,  50.62769937,  83.84292667,  94.1773086 ])

## 3 display data

In [39]:
def data_for_display(prob_data,reg_result):
    data = prob_data.drop(['density_per_km', 'latitude', 'longitude',
       'totalSnow_cm', 'sunHour', 'moon_illumination',
       'WindGustKmph', 'cloudcover', 'pressure',
       'visibility', 'winddirDegree', 'year', 'month'], axis = 1)
    
    
    data['No. of cases'] = reg_result
    
    mean_of_zika = round(data['Zika'].mean()*100)
    mean_of_nozika = round(data['No Zika'].mean()*100)
    total_no_of_cases = int(data['No. of cases'].sum())
    
    data.set_index(['date_time'], inplace=True)
    data['Zika'] = pd.Series(["{0:.0f}%".format(val * 100) for val in data['Zika']], index = data.index)
    data['No Zika'] = pd.Series(["{0:.0f}%".format(val * 100) for val in data['No Zika']], index = data.index)
    data['No. of cases'] = pd.Series(["{0:.0f}".format(val) for val in data['No. of cases']], index = data.index)
    
    
    data.rename(columns={'date_time': 'Date', 'maxtempC': 'Max temp','DewPointC': 'Dew point', 'humidity': 'Humdity', 'Precipitation': 'precipMM'}, inplace=True)

    
    return data, mean_of_zika, mean_of_nozika, total_no_of_cases

In [40]:
df = data_for_display(clf_result,reg_result)
df

(            Max temp  mintempC  Dew point  Humdity  precipMM Zika No Zika  \
 date_time                                                                   
 2016-01-28        28        15          9       49       0.2  29%     71%   
 2016-01-29        32        15         11       52       1.0  29%     71%   
 2016-01-30        30        17         14       65       2.5  32%     68%   
 2016-01-31        25        10          9       63       0.0  20%     80%   
 2016-02-01        28        11         10       65       0.0  25%     75%   
 2016-02-02        30        13         11       59       0.0  23%     77%   
 2016-02-03        33        15         12       55       0.0  30%     70%   
 2016-02-04        33        17         12       50       0.3  30%     70%   
 
            No. of cases  
 date_time                
 2016-01-28          131  
 2016-01-29           96  
 2016-01-30          125  
 2016-01-31           88  
 2016-02-01           66  
 2016-02-02           51  
 2

In [47]:
df[0]

,Max temp,mintempC,Dew point,Humdity,precipMM,Zika,No Zika,No. of cases
date_time,,,,,,,,
2016-01-28,28,15,9,49,0.2,29%,71%,131
2016-01-29,32,15,11,52,1.0,29%,71%,96
2016-01-30,30,17,14,65,2.5,32%,68%,125
2016-01-31,25,10,9,63,0.0,20%,80%,88
2016-02-01,28,11,10,65,0.0,25%,75%,66
2016-02-02,30,13,11,59,0.0,23%,77%,51
2016-02-03,33,15,12,55,0.0,30%,70%,84
2016-02-04,33,17,12,50,0.3,30%,70%,94


In [44]:
df[0].columns.values

array(['Max temp', 'mintempC', 'Dew point', 'Humdity', 'precipMM', 'Zika',
       'No Zika', 'No. of cases'], dtype=object)

In [46]:
row_data=list(df[0].values.tolist())
row_data

[[28, 15, 9, 49, 0.2, '29%', '71%', '131'],
 [32, 15, 11, 52, 1.0, '29%', '71%', '96'],
 [30, 17, 14, 65, 2.5, '32%', '68%', '125'],
 [25, 10, 9, 63, 0.0, '20%', '80%', '88'],
 [28, 11, 10, 65, 0.0, '25%', '75%', '66'],
 [30, 13, 11, 59, 0.0, '23%', '77%', '51'],
 [33, 15, 12, 55, 0.0, '30%', '70%', '84'],
 [33, 17, 12, 50, 0.3, '30%', '70%', '94']]